<a href="https://colab.research.google.com/github/OneRenix/GoBeyondColab/blob/main/Beam_College_Beam_with_Gemini_and_Google_AI_Studio_%5Bworkbook%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Beam with Gemini and Google AI Studio

This notebook shows how to use Gemini to do predictions for a stream of user prompts, using Beam ML and Google AI Studio.

To use this notebook, you will need to have a GMail account and access to Google AI Studio:
* https://aistudio.google.com/

Below in this notebook you will find instructions to generate your API key.

## Install dependencies

In [ ]:
!pip install -q -U apache-beam[interactive]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 4.2 MB/s eta 0:00:00


### Imports for using Beam with the interactive runnner

In [ ]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

from apache_beam import Pipeline

from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.ml.inference.base import ModelHandler

### Options for pipeline

In [ ]:
cli_options = ["--streaming", "--runner=InteractiveRunner"]
options = PipelineOptions(cli_options)

In [ ]:
PROMPT = """
You are an expert in geography that knows all the capitals of the world.

Whenever you are presented with a country name, you reply with its capital. For instance:

Country: France
Capital: Paris

However, if the country is Switzerland, you have to reply that it does not have an officially recognized capital. For instance:

Country: Switzerland
Capital: There is no officially recognized capital.

Just include the capital in the answer, don't include the prefix "Capital:"

Country: """

## User entries

We will present these inputs to Gemini using Beam. Again, feel free to change them to do your own experiments:

In [ ]:
user_queries = ["Spain",
                "Switzerland",
                "Argentina",
                "Japan",
                "Colombia"]

### RunInference transform

We need to write a model handler for the RunInference transform. This allows us to use any external model easily, if we have some "code snippet" that we can run to call the model and obtain a prediction.

For more details about `RunInference` and Beam ML, see
* https://beam.apache.org/documentation/ml/overview/#prediction-and-inference

For instance, using the button "Get Code" in Google AI Studio.

We store the `GEMINI_API_KEY` in a Google Colab secret, you will need to create one before you are able to use this notebook.

To get an API key, see
* https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python#setup_your_api_key
* If you are in the EU, UK or CH, check the latest status for the availability of the free tier at https://ai.google.dev/gemini-api/docs/billing#is-Gemini-free-in-EEA-UK-CH

In [ ]:
class GeminiModelHandler(ModelHandler):
  def __init__(self, api_key):
    self.api_key = api_key

  def load_model(self):
    # TODO: your code here
    pass

  def run_inference(self, batch, model, inference_args = None):
      for b in batch:
        c = model.generate_content(PROMPT + b)
        t = c.candidates[0].content.parts[0].text
        yield t

Make sure you have published your API key as a Colab secret.

See the panel on the left side to add your own secret. The name of the secret hast to be `GEMINI_API_KEY`.

In [ ]:
# Get the API key from the Google Colab secrets
from google.colab import userdata
my_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
p = Pipeline(options=options)
countries = p | beam.Create(user_queries)
# TODO: write the rest of the pipeline